In [1]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras import backend as K
from keras.layers.convolutional import *
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import  BatchNormalization
from keras import regularizers ### for SVM
import numpy as np
import tensorflow as tf
# import theano
import  matplotlib.pyplot as plt
import matplotlib
import itertools
from numpy import*
from sklearn.utils import shuffle
%matplotlib inline
import random as rn
import os
os.environ['PYTHONASHSEED']= '0'
from keras.models import load_model

tf.reset_default_graph()

Using TensorFlow backend.


In [2]:
#setting  the  seed for nummy_gennerated random numbers
np.random.seed(7)
#seting the seed for python random numbers
rn.seed(124)
#seting the seed for tensorflow random numbers
tf.set_random_seed(57)

In [3]:
train_path = '.\MAKEUP\Train' 
valid_path = '.\MAKEUP\Valid' 
#test_path = '.\CASIA1\Test' 

In [4]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (224,224),color_mode='rgb', classes = ['MakeUp','Normal'], batch_size =8)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size = (224,224),color_mode='rgb', classes = ['MakeUp','Normal'], batch_size = 8)

Found 604 images belonging to 2 classes.
Found 154 images belonging to 2 classes.


In [5]:

def add_module(input):
    #print(input.shape)
    
    Conv2D_reduce = Conv2D(16, (1,1), strides=(2,2), activation='relu', padding='same')(input)
    #print(Conv2D_reduce.shape)
    
    Conv2D_1_1 = Conv2D(16, (1,1), activation='relu', padding='same')(input)
    #print(Conv2D_1_1.shape)
    Conv2D_3_3 = Conv2D(16, (3,3),strides=(2,2), activation='relu', padding='same')(Conv2D_1_1)
    #print(Conv2D_3_3.shape)
    Conv2D_5_5 = Conv2D(16, (5,5),strides=(2,2), activation='relu', padding='same')(Conv2D_1_1)
    #print(Conv2D_5_5.shape)
    
    MaxPool2D_3_3 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(input)
    #print(MaxPool2D_3_3.shape)
    Cov2D_Pool = Conv2D(16, (1,1), activation='relu', padding='same')(MaxPool2D_3_3)
    #print(Cov2D_Pool.shape)
    
    concat = Concatenate(axis=-1)([Conv2D_reduce,Conv2D_3_3,Conv2D_5_5,Cov2D_Pool])
    #print(concat.shape)
    
    return concat

In [6]:
# GoogLeNet Define, not using the sequential model, becuase googlenet is not sequential
num_classes = 2
input = Input(shape=(224, 224, 3))

Conv2D_1 = Conv2D(64, (3,3), activation='relu', padding='same')(input)
MaxPool2D_1 = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(Conv2D_1)
BatchNorm_1 = BatchNormalization()(MaxPool2D_1)

Module_1 = add_module(BatchNorm_1)
Module_1 = add_module(Module_1)

Output = Flatten()(Module_1)
Output = Dense(num_classes, activation='softmax')(Output)

In [7]:
model = Model(inputs=[input], outputs=[Output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (

In [8]:
model.layers.pop()
last = model.layers[-1].output
x = Dense(2, activation = 'linear', kernel_regularizer=regularizers.l2(0.001))(last)
finetuned_model = Model(model.input, x)

In [9]:
finetuned_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (

In [10]:
n = len(finetuned_model.layers)
for idx, layer in enumerate(finetuned_model.layers):
    if idx < (n-2): 
           layer.trainable = True 
    else:
           layer.trainable = True   

In [ ]:
opt = Adam(lr=0.001, decay=10e-6)
finetuned_model.compile(loss='hinge', optimizer=opt, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
# Save check point
filepath = "weights.GoogLeNet.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose =1, save_best_only = True, mode ='max')
callbacks_list= [checkpoint]
 # Fit the model
# model.fit(X,Y, validation_split=0.33, nb_epoch =150, batch_size =10, callbacks = callbacks_list, verbose =0)

history = finetuned_model.fit_generator(train_batches, steps_per_epoch = 120, validation_data = valid_batches, validation_steps=26, epochs =200, shuffle=True, callbacks = callbacks_list, verbose =2)

Epoch 1/200
 - 15s - loss: 0.5064 - acc: 0.4729 - val_loss: 0.5039 - val_acc: 0.5594

Epoch 00001: val_acc improved from -inf to 0.55941, saving model to weights.GoogLeNet.best.hdf5
Epoch 2/200
 - 8s - loss: 0.5034 - acc: 0.4979 - val_loss: 0.5030 - val_acc: 0.5644

Epoch 00002: val_acc improved from 0.55941 to 0.56436, saving model to weights.GoogLeNet.best.hdf5
Epoch 3/200
 - 8s - loss: 0.5027 - acc: 0.5062 - val_loss: 0.5024 - val_acc: 0.5198

Epoch 00003: val_acc did not improve
Epoch 4/200
 - 8s - loss: 0.5022 - acc: 0.5042 - val_loss: 0.5019 - val_acc: 0.5050

Epoch 00004: val_acc did not improve
Epoch 5/200
 - 7s - loss: 0.5017 - acc: 0.5021 - val_loss: 0.5016 - val_acc: 0.4950

Epoch 00005: val_acc did not improve
Epoch 6/200
 - 7s - loss: 0.5014 - acc: 0.5021 - val_loss: 0.5013 - val_acc: 0.4703

Epoch 00006: val_acc did not improve
Epoch 7/200
 - 7s - loss: 0.5012 - acc: 0.5104 - val_loss: 0.5010 - val_acc: 0.4901

Epoch 00007: val_acc did not improve
Epoch 8/200
 - 7s - loss